## Installing relevant libraries

In [8]:
!pip install --upgrade langchain

In [9]:
!pip install langchain langchain_community langchain_google_genai



---



## Importing Libraries

In [23]:
from google.colab import userdata
import os
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
#from langchain.agents import ToolAgent, ToolAgentExecutor

from langchain.agents import AgentExecutor, create_tool_calling_agent

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [34]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,

)

In [35]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

In [36]:
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

In [21]:
# Define tools
tools = [calculator]

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the calculator tool for calculation.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [38]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [45]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
#agent_executor.invoke({"input": "what is LangChain?"})

In [46]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
You told me your name is Bob.

> Finished chain.


{'input': "what's my name? Don't use tools to look this up unless you NEED to",
 'chat_history': [HumanMessage(content='hi! my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'You told me your name is Bob.'}

In [49]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hi Bob! It's nice to meet you. How can I help you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': "Hi Bob! It's nice to meet you. How can I help you today?"}

In [51]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [52]:
message_history = ChatMessageHistory()

In [53]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [54]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I help you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I help you today?'}

In [60]:
agent_with_chat_history.invoke(
    {"input": "hi"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).